In [5]:
%%sh
cat > pi-mpi.c << EOF
#include <mpi.h>
#include <stdio.h>
#include <time.h>
 
double funkcja(double x)
{
  return x*x;
}

int main(int argc, char **argv)
{
    int numer_procesu;
    int liczba_procesow;
    int tag=1;
    
    MPI_Status status;
    MPI_Init(&argc, &argv);
    MPI_Comm_rank(MPI_COMM_WORLD, &numer_procesu);
    MPI_Comm_size(MPI_COMM_WORLD, &liczba_procesow);
    clock_t t;
    double czas_potrzebny;

    double l_procesow = liczba_procesow, n_procesu = numer_procesu;

    double granica_dolna_calki = 1,granica_gorna_calki = 4;
    double h = (granica_gorna_calki - granica_dolna_calki) / l_procesow;
    
    if(numer_procesu == 0)
    {
      double calka = 0 ,x=0;
      for(int i=1; i<liczba_procesow; i++)
      {
        MPI_Recv(&x, 1, MPI_DOUBLE, i, tag, MPI_COMM_WORLD, &status);
        calka +=x;
      }
      printf("Wartosc funkcji dolnej granicy calki= %f \n",funkcja(granica_dolna_calki)/2 * h);
      printf("Wartosc funkcji gornej granicy calki= %f \n",funkcja(granica_gorna_calki)/2 * h);

      printf("Wartosc calki = %1f \n", h * (funkcja(granica_dolna_calki)/2 + calka 
        + funkcja(granica_gorna_calki)/2));
    }

    if(numer_procesu != 0)
    {
        t = clock();
        double xi,wynik;
        n_procesu = numer_procesu;
        xi = granica_dolna_calki + (n_procesu/l_procesow) * (granica_gorna_calki - granica_dolna_calki);
        wynik = funkcja(xi);
        printf("Wynik dla procesu %d to = %f \t", numer_procesu,wynik*h);
        MPI_Send(&wynik, 1, MPI_DOUBLE, 0, tag, MPI_COMM_WORLD);
        t = clock() - t;
        czas_potrzebny = ((double)t)/CLOCKS_PER_SEC;
        printf("Czas potrzebny do wykonania procesu numer %d to = %f\n",numer_procesu,czas_potrzebny);
    }

    MPI_Finalize();
}
EOF
mpicc pi-mpi.c && mpirun -n 10 --allow-run-as-root a.out

Wynik dla procesu 1 to = 0.507000 	Czas potrzebny do wykonania procesu numer 1 to = 0.000085
Wynik dla procesu 6 to = 2.352000 	Czas potrzebny do wykonania procesu numer 6 to = 0.000051
Wynik dla procesu 3 to = 1.083000 	Czas potrzebny do wykonania procesu numer 3 to = 0.000044
Wynik dla procesu 4 to = 1.452000 	Czas potrzebny do wykonania procesu numer 4 to = 0.000058
Wynik dla procesu 7 to = 2.883000 	Czas potrzebny do wykonania procesu numer 7 to = 0.000062
Wynik dla procesu 9 to = 4.107000 	Czas potrzebny do wykonania procesu numer 9 to = 0.000052
Wynik dla procesu 8 to = 3.468000 	Czas potrzebny do wykonania procesu numer 8 to = 0.000058
Wynik dla procesu 5 to = 1.875000 	Czas potrzebny do wykonania procesu numer 5 to = 0.000037
Wynik dla procesu 2 to = 0.768000 	Czas potrzebny do wykonania procesu numer 2 to = 0.000032
Wartosc funkcji dolnej granicy calki= 0.150000 
Wartosc funkcji gornej granicy calki= 2.400000 
Wartosc calki = 21.045000 
